In [1]:
import pdfplumber

In [2]:
pdf_path = "PdfNAP (1).pdf"  # Replace with your PDF file path

In [3]:
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
lines = text.splitlines()

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [4]:
lines

['Offert 2',
 'Datum: 2025-02-25',
 'Vår referens: Erik Svensson',
 'Er referens: Maria Lindgren',
 'Kund: Sapa Fönster AB, Vetlanda',
 'Aluminiumprofiler från Nordisk Alu Profil/ Purso OY',
 'Profil nr / Vikt Längd/m Kap + truml ca antal Pris Legerin',
 'Kund ref. kg/m m Pris/st Årsvolym st kr/st g:',
 'SEK',
 'Karmlist 1,342 23,8 0,78 40000 2,92 Rå',
 'Karmlist 1,342 25,8 0,78 40000 3,05 Rå',
 'Karmlist 1,342 23,8 0,78 85000 2,88 Rå',
 'Karmlist 1,342 25,8 0,78 85000 3,01 Rå',
 'Verktygskostnad: 14500kr',
 'Legering: Rå',
 'Toleranser: EN 755-9',
 'Ytbehandling: EN-AW-6063-T5',
 'Lev. längd: Längder enligt ovan',
 'Lev. villkor: Ex Works Halmstad',
 'Lev. tid: första 8-10 veckor från order därefter 5-6 veckor',
 'NOT: Minst 15000 bitar kapade, ok att blanda längder, inkl trumling',
 'Betalningsvillkor: 30 dagar netto. Dröjsmålsränta 7,5%',
 'Giltighet: Offererade priser gäller fast för leveranser t.o.m. 2025-05-30',
 'Allmänna villkor: NAPFV2017',
 'Råvara: 3,4 Euro / kg']

In [ ]:
# Identify the start of the table and the end (marked by "Verktygskostnad:")
idx_tool = None
idx_table_start = None
for i, line in enumerate(lines):
    if idx_table_start is None and ("Profil nr" in line or (line.strip().startswith("Profil") and "kg/m" in line)):
        idx_table_start = i
    if line.strip().startswith("Verktygskostnad:"):
        idx_tool = i
        break

if idx_table_start is None or idx_tool is None:
    data_lines = []  # assign empty if parsing failed
else:
    #print(f"Table starts at line {idx_table_start}, ends at line {idx_tool}")

    # Determine where the actual data lines begin (after the header lines)
    idx_data_start = idx_table_start
    for j in range(idx_table_start, idx_tool):
        # The last header line often contains "Årsvolym" or "SEK" or ends with "g:"
        if "Årsvolym" in lines[j] or lines[j].strip() == "SEK" or lines[j].strip().endswith("g:"):
            idx_data_start = j + 1

    # Extract table data lines, filtering out header artifacts
    data_lines = []
    for k in range(idx_data_start, idx_tool):
        line_clean = lines[k].strip()
        if not line_clean or any(line_clean.startswith(h) for h in ["Profil", "Kund ref.", "Vikt", "Pris/st", "Kap", "SEK"]):
            continue

        parts = line_clean.split()
        if len(parts) >= 7:
            # We assume last 6 parts are numeric values, first is profile name
            profile_name = " ".join(parts[:-6])
            row = [
                profile_name,
                parts[-6],  # Vikt
                parts[-5],  # Längd
                parts[-4],  # Kap+truml
                parts[-3],  # Årsvolym
                parts[-2],  # Pris
                parts[-1],  # Legering
            ]
            data_lines.append(row)   

# Determine where the actual data lines begin (after the header lines)
idx_data_start = idx_table_start
for j in range(idx_table_start, idx_tool):
    # The last header line often contains "Årsvolym" or "SEK" or ends with "g:"
    if "Årsvolym" in lines[j] or lines[j].strip() == "SEK" or lines[j].strip().endswith("g:"):
        idx_data_start = j + 1

# Extract table data lines, filtering out header artifacts
data_lines = []
for k in range(idx_data_start, idx_tool):
    line_clean = lines[k].strip()
    if not line_clean or any(line_clean.startswith(h) for h in ["Profil", "Kund ref.", "Vikt", "Pris/st", "Kap", "SEK"]):
        continue

    parts = line_clean.split()
    if len(parts) >= 7:
        # We assume last 6 parts are numeric values, first is profile name
        profile_name = " ".join(parts[:-6])
        row = [
            profile_name,
            parts[-6],  # Vikt
            parts[-5],  # Längd
            parts[-4],  # Kap+truml
            parts[-3],  # Årsvolym
            parts[-2],  # Pris
            parts[-1],  # Legering
        ]
        data_lines.append(row)   
    
#print("Data lines extracted:", data_lines)